# 2022-01-19e Planning for ARC4 combined fits

Results below are for commit: https://github.com/gmarkkula/COMMOTIONSFramework/commit/651779306723ef01690536e2b15b013a2647d6b0

(Rerunning the code below for other commits is not likely/guaranteed to give the same results.)

First small tests on ARC4 are confirming that the combined fits now run at the hoped-for speed on ARC4, with not a massive difference between `oVA` and `oVAa` models - 6 s per parameterisation seems a sensible estimate for both, to be on the safe side. 

The code below goes through the retained deterministic and probabilistic parameterisations below to estimate total run time on ARC4.

My conclusion is that I will divide into four separate `do_8<a-d>.sh` jobs, one for each retained deterministic model. With max 5000 parameterisations per combined model, each such job should be possible to conclude within 48 h, as shown below. I am including those bash files with this commit.

In [10]:
# append root folder of repo to PYTHONPATH (https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder)
import sys
from pathlib import Path
parent_path = str(Path('.').absolute().parent)
if not (parent_path in sys.path):
    sys.path.append(parent_path)
    sys.path.append(parent_path + '/SCPaper')
print(sys.path)

import math
import pickle
import sc_fitting

INCL_DET_MODELS = 'all' # ('oVAaoVAloBEvoAI', 'oVAoVAloBEvoAI') # either 'all' or a tuple of names of models to include
EXCL_PROB_MODELS = ('oVAoAN', 'oVAoEAoAN')
MAX_PARAMETERISATIONS = 5000
TIME_PER_PARAMETERISATION = 6 # s

# load the retained models
with open(sc_fitting.FIT_RESULTS_FOLDER + '/'
          + sc_fitting.RETAINED_DET_FNAME, 'rb') as file_obj:
    det_models = pickle.load(file_obj)
with open(sc_fitting.FIT_RESULTS_FOLDER + '/' 
          + sc_fitting.RETAINED_PROB_FNAME, 'rb') as file_obj:
    prob_models = pickle.load(file_obj)

# loop through all combinations of retained deterministic and 
# probabilistic models
total_fit_time = 0
for det_model in det_models:
    
    det_total_fit_time = 0
    print(f'*** Deterministic model {det_model.model} ***\n')

    # this deterministic model included?
    if INCL_DET_MODELS != 'all':
        assert(type(INCL_DET_MODELS) is tuple)
        if not det_model.model in INCL_DET_MODELS:
            continue
    for prob_model in prob_models:

        # this probabilistic model excluded?
        if prob_model.model in EXCL_PROB_MODELS:
            continue

        # get combined model name
        assert(prob_model.model[0:3] == 'oVA')
        model_name = det_model.model + prob_model.model[3:]

        # get the combined list of parameterisations
        # - get number of parameters and retained parameterisations
        n_det_params = len(det_model.param_names)
        n_prob_params = len(prob_model.param_names)
        n_det_parameterisations = det_model.params_array.shape[0]
        n_prob_parameterisations = prob_model.params_array.shape[0]
        n_comb_parameterisations = (n_det_parameterisations 
                                    * n_prob_parameterisations)
        n_act_comb_parameterisations = min(n_comb_parameterisations, MAX_PARAMETERISATIONS)
        
        print(f'\tModel {model_name}: {n_det_parameterisations}'
              f' x {n_prob_parameterisations} = {n_comb_parameterisations}'
              f' --> {n_act_comb_parameterisations} combined parameterisations...')
        fit_time = n_act_comb_parameterisations * TIME_PER_PARAMETERISATION
        print(f'\t\tWill take {fit_time} s ≈ {fit_time / 3600:.1f} h\n')
        det_total_fit_time += fit_time
        total_fit_time += fit_time

    print(f'Total fit time {det_total_fit_time} s ≈ {det_total_fit_time / 3600:.1f} h\n')
        
print(f'\n**********\nGrand total fit time {total_fit_time} s ≈ {total_fit_time / 3600:.1f} h')    

['C:\\GITHUB\\COMMOTIONSFramework\\diary notebooks', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\python39.zip', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\DLLs', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\lib', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64', '', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\lib\\site-packages', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\lib\\site-packages\\win32', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\lib\\site-packages\\win32\\lib', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\lib\\site-packages\\Pythonwin', 'C:\\WinPython\\WPy64-3920\\python-3.9.2.amd64\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\tragma\\.ipython', 'C:\\GITHUB\\COMMOTIONSFramework', 'C:\\GITHUB\\COMMOTIONSFramework/SCPaper']
*** Deterministic model oVAaoVAloBEooBEvoAI ***

	Model oVAaoVAloBEooBEvoAIoEAoSNc: 5000 x 55 = 275000 --> 5000 combined parameterisations...
		Will take 30000 s ≈ 8.3 h

	Model oVAaoVAloBEooBEvoAIoEAoSNcoPF: 5000 x 1